In [30]:
import json
import correctionlib
import numpy as np
import awkward as ak
from coffea import processor
from coffea.analysis_tools import Weights

In [31]:
"""
CorrectionLib files are available from: /cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration - synced daily
"""
pog_correction_path = "/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/"
pog_jsons = {
    "pileup": ["LUM", "puWeights.json.gz"],
}

def get_UL_year(year):
    if year == "2016":
        year = "2016postVFP"
    elif year == "2016APV":
        year = "2016preVFP"
    return f"{year}_UL"

def get_pog_json(obj, year):
    try:
        pog_json = pog_jsons[obj]
    except:
        print(f'No json for {obj}')
    year = get_UL_year(year)
    return f"{pog_correction_path}POG/{pog_json[0]}/{year}/{pog_json[1]}"

def add_pileup_weight(weights, year, mod, nPU):
    """
    https://cms-nanoaod-integration.web.cern.ch/commonJSONSFs/LUMI_puWeights_Run2_UL/
    """
    cset = correctionlib.CorrectionSet.from_file(get_pog_json("pileup", year + mod))

    year_to_corr = {
        '2016': 'Collisions16_UltraLegacy_goldenJSON',
        '2017': 'Collisions17_UltraLegacy_goldenJSON',
        '2018': 'Collisions18_UltraLegacy_goldenJSON',
    }

    values = {}
    values["nominal"] = cset[year_to_corr[year]].evaluate(nPU, "nominal")
    values["up"] = cset[year_to_corr[year]].evaluate(nPU, "up")
    values["down"] = cset[year_to_corr[year]].evaluate(nPU, "down")

    weights.add("pileup", values["nominal"], values["up"], values["down"])

In [32]:
class BackgroundEstimatorProcessor(processor.ProcessorABC):
    def __init__(
        self, 
        year: str = "2017", 
        yearmod: str = "",
    ):
        self._year = year
        self._yearmod = yearmod
        
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events: ak.Array):
        """Returns skimmed events which pass preselection cuts and with the branches listed in self._skimvars"""  
        # weights initializer
        self.weights = Weights(len(events), storeIndividual=True)
        
        # save weights for MC data
        self.isMC = hasattr(events, "genWeight")
        if self.isMC:
            # gen weight
            self.weights.add("genweight", events.genWeight)
            
            # l1prefiring weight
            # WHAT ABOUT 2018? 
            if self._year in ("2016", "2017"):
                self.weights.add(
                    "L1Prefiring", events.L1PreFiringWeight.Nom, events.L1PreFiringWeight.Up, events.L1PreFiringWeight.Dn
                )
                
            # pileup weight
            add_pileup_weight(self.weights, self._year, self._yearmod, nPU=ak.to_numpy(events.Pileup.nPU))
            
            weigths = {}
            weigths["genweight"] = self.weights.partial_weight(include=["genweight"])
            weigths["L1Prefiring"] = self.weights.partial_weight(include=["L1Prefiring"])
            weigths["pileup"] = self.weights.partial_weight(include=["pileup"])
            weigths["common"] = self.weights.partial_weight(include=["genweight", "L1Prefiring", "pileup"])
            
        return weigths
            
    def postprocess(self, accumulator):
        return accumulator

In [33]:
fileset = {
    "DYJetsToLL_Pt-100To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8": ["root://xcache//store/mc/RunIISummer20UL17NanoAODv2/DYJetsToLL_Pt-100To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v8-v1/240000/A3975393-1AED-B547-8CA2-28B5841C4895.root"],
}

In [34]:
out = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=BackgroundEstimatorProcessor(),
    executor=processor.IterativeExecutor,
    executor_args={
        "schema": processor.NanoAODSchema,
    },
)

Output()

In [35]:
out

{'genweight': array([1137.05895996, 1137.05895996, 3411.17687988, ..., 1137.05895996,
        1137.05895996, 2274.11791992]),
 'common': array([ 935.65991067,  895.94746514, 3218.94532182, ...,  989.14313957,
         841.57688582, 2101.3717697 ]),
 'L1Prefiring': array([4.87387884, 5.39471781, 5.72314876, ..., 5.80820811, 5.39434707,
        4.92705172]),
 'pileup': array([5.88948196, 5.75624723, 5.94009577, ..., 6.13647971, 5.93715912,
        6.12800179])}